In [444]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib


import os,sys
from importlib import reload
import copy
from collections import namedtuple

sys.path.append("../tracker")

In [445]:
import kalmanfilter as KF
import utilities as Util
import datatypes
import vertexfinder as VF

reload(VF)
reload(TF)
reload(Util)

<module 'utilities' from '../tracker\\utilities.py'>

In [497]:
tracks=joblib.load("tracks_example.joblib")
# tracks

In [498]:
vf = VF.VertexFitter(debug=True)
vertices = vf.run(tracks)
# %timeit -n 2 vf.run(tracks)

VertexSeed(x0=-254.3629291916415, y0=9137.003168896139, z0=13024.971865263946, t0=13.227374726695434, cov=0, chi2=17.81734158533356, dist=216.02999341565217, Ntracks=3, trackind1=0, trackind2=2)
VertexSeed(x0=-351.4531698511656, y0=9357.807456194188, z0=13059.608654484686, t0=14.392923494031885, cov=0, chi2=12.583598314528553, dist=237.34679919726057, Ntracks=3, trackind1=2, trackind2=4)
VertexSeed(x0=-257.29986992472266, y0=9356.669250960116, z0=13084.121651938869, t0=15.206390448987545, cov=0, chi2=9.427186073011585, dist=89.81524976894703, Ntracks=3, trackind1=1, trackind2=2)
VertexSeed(x0=-451.4407245868403, y0=9480.181999224467, z0=13109.600118731594, t0=15.569468424528504, cov=0, chi2=3.8017866755224805, dist=71.33984011876291, Ntracks=3, trackind1=3, trackind2=4)
--- New seed for vertex --- 
  Seed VertexSeed(x0=-451.4407245868403, y0=9480.181999224467, z0=13109.600118731594, t0=15.569468424528504, cov=0, chi2=3.8017866755224805, dist=71.33984011876291, Ntracks=3, trackind1=3, t

In [496]:
0.01*(13048.991219115298-7000-4950), 0.01*(-232.8637214423652), 0.01*(9274.048555922185-8547) 

(10.98991219115298, -2.3286372144236522, 7.270485559221852)

In [499]:
print(len(vertices))
print(len(vertices[0].tracks))
print(vertices[0])

1
3
Vertex(x0=-337.91196167006274, y0=8830.78767243668, z0=13065.563220776457, t0=-499.99998997435654, cov=array([[ 3.58389094e+03, -1.19147879e+03, -7.47309049e+02,
        -3.37105249e-04],
       [-1.19147879e+03,  1.02782078e+03,  5.97217552e+02,
         2.89592212e-03],
       [-7.47309049e+02,  5.97217552e+02,  4.21963715e+02,
         1.31529056e-04],
       [-3.37105249e-04,  2.89592212e-03,  1.31529056e-04,
         5.91736319e-04]]), chi2=176.88818672334136, tracks=[Track(x0=-379.19783442375007, y0=9894.0, z0=13414.311791442962, t0=33.2565541322419, Ax=0.174833321035048, Ay=1, Az=0.7503867059983871, At=0.04617355224278653, cov=array([[ 2.07179674e+00, -3.30205612e-02,  4.96246842e-02,
        -1.20511596e-01,  1.96994341e-02, -7.27034327e-06],
       [-3.30205612e-02,  5.80862671e+00, -2.68555659e-01,
         2.92922923e-03, -3.92885441e-02,  3.55725489e-03],
       [ 4.96246842e-02, -2.68555659e-01,  1.43398990e+00,
        -3.56848874e-05,  8.61175050e-02, -1.16593815e-01

In [ ]:
def root_hits_extractor_tracks(Tree, entry):
    Tree.GetEntry(entry)
    c2list = lambda x: np.array([x.at(i) for i in range(x.size())])

    Digi_layer = c2list(Tree.Digi_layer_id)
    # Only select hits in tracking layer
    mask = (Digi_layer>1) & (Digi_layer<=5) 

    Digi_x = c2list(Tree.Digi_x)[mask]
    Digi_y = c2list(Tree.Digi_y)[mask]
    Digi_z = c2list(Tree.Digi_z)[mask]
    Digi_t = c2list(Tree.Digi_time)[mask]
    Digi_track_id = c2list(Tree.Digi_track_id)[mask]
    Digi_layer = Digi_layer[mask]
    Hits = Util.hit.make_hits(Digi_x, Digi_y, Digi_z, Digi_t, Digi_layer)

    track_ids = np.unique(Digi_track_id)
    track_hits  = {}
    for trackid in track_ids:
        mask = Digi_track_id==trackid
        hit_group = []
        for i in np.flatnonzero(mask):
            hit_group.append(Hits[i])
        track_hits[int(trackid)] = hit_group

    return track_hits